# Demo for $\mathbf{Z}_3 \wr \mathbf{Z}_3 \wr \mathbf{Z}_3$

In [1]:
import itertools
import pickle
import random

import numpy as np
import scipy
from tqdm.auto import tqdm

from wrepy import (
    ArrayWrPermutation,
    CyclicGroupPermutationFactory,
    GeneratorSetFactory,
    Permutation,
    PermutationGroup,
    Portrait,
)

## Util functions

In [2]:
def column_print(p: Permutation):
    for k, v in sorted(p.rule.items()):
        print(f"{k} -> {v}")

In [28]:
def get_cycle(p: Permutation, v: tuple[int, int, int]) -> list[tuple[int, int, int]]:
    cycle = []
    while v not in cycle:
        cycle.append(v)
        v = v ** p
    return cycle

## Data preparation

In [3]:
N = 3
z3 = PermutationGroup(
    set(range(N)),  # define the underling set as {0, 1, 2}
    CyclicGroupPermutationFactory,  # setting the rule how to build group from underling set
)
z3z3 = z3.wreath_product(z3)
z3z3z3 = z3z3.wreath_product(z3)
z3z3z3

PermutationGroup(order=1594323)

In [4]:
# eagerly collecting all elements
elements = list(tqdm(z3z3z3.elements))

  0%|          | 0/1594323 [00:00<?, ?it/s]

In [5]:
order_3_elements_mask = np.load(
    "../z3z3z3_order_3.npy"
)  # a mask for each element of group True if the element has order 3
# applying a mask to elements
elements = list(itertools.compress(elements, order_3_elements_mask))
len(elements)

104246

In [6]:
alpha = list(z3.elements)
alpha = [alpha[-1], alpha[0], alpha[1]]

In [7]:
result_index = scipy.sparse.load_npz("../sparse_res.npz")
valid_pairs_ids = result_index.nonzero()

In [8]:
result_index

<104246x104246 sparse matrix of type '<class 'numpy.bool_'>'
	with 40223304 stored elements in Compressed Sparse Column format>

In [9]:
def get_pair(index: int) -> tuple[Permutation, Permutation]:
    """Returns a C, D pair of generator elements"""
    c_idx = valid_pairs_ids[0][index]
    d_idx = valid_pairs_ids[1][index]

    C = elements[c_idx]
    D = elements[d_idx]
    return C, D

## Demo

In [73]:
# Select a random pair C, D
pair_index = random.randrange(0, result_index.size)
print("Selected pair:",pair_index)
C, D = get_pair(pair_index)

Selected pair: 39677726


In [74]:
# Check order of C
C.order

3

In [75]:
# Check order of D
D.order

3

In [76]:
# Full view of C
column_print(C)

(0, 0, 0) -> (0, 2, 2)
(0, 0, 1) -> (0, 2, 0)
(0, 0, 2) -> (0, 2, 1)
(0, 1, 0) -> (0, 0, 0)
(0, 1, 1) -> (0, 0, 1)
(0, 1, 2) -> (0, 0, 2)
(0, 2, 0) -> (0, 1, 1)
(0, 2, 1) -> (0, 1, 2)
(0, 2, 2) -> (0, 1, 0)
(1, 0, 0) -> (1, 1, 1)
(1, 0, 1) -> (1, 1, 2)
(1, 0, 2) -> (1, 1, 0)
(1, 1, 0) -> (1, 2, 2)
(1, 1, 1) -> (1, 2, 0)
(1, 1, 2) -> (1, 2, 1)
(1, 2, 0) -> (1, 0, 0)
(1, 2, 1) -> (1, 0, 1)
(1, 2, 2) -> (1, 0, 2)
(2, 0, 0) -> (2, 0, 1)
(2, 0, 1) -> (2, 0, 2)
(2, 0, 2) -> (2, 0, 0)
(2, 1, 0) -> (2, 1, 0)
(2, 1, 1) -> (2, 1, 1)
(2, 1, 2) -> (2, 1, 2)
(2, 2, 0) -> (2, 2, 2)
(2, 2, 1) -> (2, 2, 0)
(2, 2, 2) -> (2, 2, 1)


In [77]:
# Full view of D
column_print(D)

(0, 0, 0) -> (0, 0, 2)
(0, 0, 1) -> (0, 0, 0)
(0, 0, 2) -> (0, 0, 1)
(0, 1, 0) -> (0, 1, 1)
(0, 1, 1) -> (0, 1, 2)
(0, 1, 2) -> (0, 1, 0)
(0, 2, 0) -> (0, 2, 1)
(0, 2, 1) -> (0, 2, 2)
(0, 2, 2) -> (0, 2, 0)
(1, 0, 0) -> (1, 2, 2)
(1, 0, 1) -> (1, 2, 0)
(1, 0, 2) -> (1, 2, 1)
(1, 1, 0) -> (1, 0, 2)
(1, 1, 1) -> (1, 0, 0)
(1, 1, 2) -> (1, 0, 1)
(1, 2, 0) -> (1, 1, 2)
(1, 2, 1) -> (1, 1, 0)
(1, 2, 2) -> (1, 1, 1)
(2, 0, 0) -> (2, 2, 0)
(2, 0, 1) -> (2, 2, 1)
(2, 0, 2) -> (2, 2, 2)
(2, 1, 0) -> (2, 0, 1)
(2, 1, 1) -> (2, 0, 2)
(2, 1, 2) -> (2, 0, 0)
(2, 2, 0) -> (2, 1, 2)
(2, 2, 1) -> (2, 1, 0)
(2, 2, 2) -> (2, 1, 1)


In [78]:
C.fixed_points()

[(2, 1, 1), (2, 1, 2), (2, 1, 0)]

In [79]:
D.fixed_points()

[]

In [80]:
cyc_1 = get_cycle(D, C.fixed_points()[0])
cyc_1

[(2, 1, 1), (2, 0, 2), (2, 2, 2)]

In [81]:
cyc_2 = get_cycle(D, C.fixed_points()[1])
cyc_2

[(2, 1, 2), (2, 0, 0), (2, 2, 0)]

In [82]:
cyc_3 = get_cycle(D, C.fixed_points()[2])
cyc_3

[(2, 1, 0), (2, 0, 1), (2, 2, 1)]

In [83]:
# fixed points of C are in the different cycles of D
set(cyc_1).intersection(set(cyc_2)).intersection(set(cyc_3))

set()

In [84]:
# conjugation check
D * (C.inverse() * D * C) == (C.inverse() * D * C) * D

True

In [85]:
#  (d * c ^2) ^ 3
# check if not trivial
# Note: the ** power operation is reserved for action of permutation on a point (see: get_cycle fn on top)
dc23 = (D * (C * C)) * (D * (C * C)) * (D * (C * C))
column_print(dc23)

print("---- Non trival elements ----")
for k, v in sorted(dc23.rule.items()):
    if k != v:
        print(f"{k} -> {v}")

(0, 0, 0) -> (0, 0, 1)
(0, 0, 1) -> (0, 0, 2)
(0, 0, 2) -> (0, 0, 0)
(0, 1, 0) -> (0, 1, 1)
(0, 1, 1) -> (0, 1, 2)
(0, 1, 2) -> (0, 1, 0)
(0, 2, 0) -> (0, 2, 1)
(0, 2, 1) -> (0, 2, 2)
(0, 2, 2) -> (0, 2, 0)
(1, 0, 0) -> (1, 0, 0)
(1, 0, 1) -> (1, 0, 1)
(1, 0, 2) -> (1, 0, 2)
(1, 1, 0) -> (1, 1, 0)
(1, 1, 1) -> (1, 1, 1)
(1, 1, 2) -> (1, 1, 2)
(1, 2, 0) -> (1, 2, 0)
(1, 2, 1) -> (1, 2, 1)
(1, 2, 2) -> (1, 2, 2)
(2, 0, 0) -> (2, 0, 0)
(2, 0, 1) -> (2, 0, 1)
(2, 0, 2) -> (2, 0, 2)
(2, 1, 0) -> (2, 1, 0)
(2, 1, 1) -> (2, 1, 1)
(2, 1, 2) -> (2, 1, 2)
(2, 2, 0) -> (2, 2, 0)
(2, 2, 1) -> (2, 2, 1)
(2, 2, 2) -> (2, 2, 2)
---- Non trival elements ----
(0, 0, 0) -> (0, 0, 1)
(0, 0, 1) -> (0, 0, 2)
(0, 0, 2) -> (0, 0, 0)
(0, 1, 0) -> (0, 1, 1)
(0, 1, 1) -> (0, 1, 2)
(0, 1, 2) -> (0, 1, 0)
(0, 2, 0) -> (0, 2, 1)
(0, 2, 1) -> (0, 2, 2)
(0, 2, 2) -> (0, 2, 0)


In [86]:
# check group order
target_group = PermutationGroup(
    z3z3z3.underlying_set,
    GeneratorSetFactory,
    generator_set=([C.rule, D.rule]),
)
target_group

/home/ailadin/Projects/permutation_groups/wrepy/main.py:196: UserWarning: Current implementation of generator set factory uses a lot of memory & doen't stop for infinite group.
  warn(


PermutationGroup(order=81)

In [87]:
# The elements of a relust group can also be examied if required

target_elements = list(target_group.elements)
column_print(target_elements[80])

(0, 0, 0) -> (0, 1, 0)
(0, 0, 1) -> (0, 1, 1)
(0, 0, 2) -> (0, 1, 2)
(0, 1, 0) -> (0, 2, 2)
(0, 1, 1) -> (0, 2, 0)
(0, 1, 2) -> (0, 2, 1)
(0, 2, 0) -> (0, 0, 0)
(0, 2, 1) -> (0, 0, 1)
(0, 2, 2) -> (0, 0, 2)
(1, 0, 0) -> (1, 0, 1)
(1, 0, 1) -> (1, 0, 2)
(1, 0, 2) -> (1, 0, 0)
(1, 1, 0) -> (1, 1, 0)
(1, 1, 1) -> (1, 1, 1)
(1, 1, 2) -> (1, 1, 2)
(1, 2, 0) -> (1, 2, 2)
(1, 2, 1) -> (1, 2, 0)
(1, 2, 2) -> (1, 2, 1)
(2, 0, 0) -> (2, 1, 1)
(2, 0, 1) -> (2, 1, 2)
(2, 0, 2) -> (2, 1, 0)
(2, 1, 0) -> (2, 2, 1)
(2, 1, 1) -> (2, 2, 2)
(2, 1, 2) -> (2, 2, 0)
(2, 2, 0) -> (2, 0, 1)
(2, 2, 1) -> (2, 0, 2)
(2, 2, 2) -> (2, 0, 0)
